<a href="https://colab.research.google.com/github/ahfat/lc-chatgpt4/blob/dev/exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install cohere tiktoken openai

!pip install langchain

In [3]:
!pip install python-dotenv

In [4]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

In [5]:
_ = load_dotenv(find_dotenv())
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

file_name = "conference_session_info.csv"

df = pd.read_csv(file_name)
df.shape

(30, 8)

In [7]:
df.head()

,Start Date,End Date,Session Name,Session Description,Session Track,Industry,Speaker Name,Room Name
0,07/27/2020 02:00 PM,07/27/2020 03:30 PM,3D Printing for the Non-Tech Minded,This is 3D Printing 101 for those in the makerspace that don’t consider themselves technically astute.,3D Printing and Design,Technology,Jeffery Lowe & Marysa Balma,Room 101
1,07/27/2020 02:00 PM,07/27/2020 03:00 PM,3D Printing with Clay,"Clay has historically been a hands-on medium for over 20,000 years, both to create practical items for day-to-day living, and art for day-to-day beauty. Now with the advent of commercially available 3D clay printers, artists and engineers alike are creating inspirational pieces that were previously unimaginable.",3D Printing and Design,Education,Julie Parker,Room 201
2,07/27/2020 02:00 PM,07/27/2020 03:30 PM,Art in the Age of Automation,"There are some people who don’t believe that art can be “art” if it is made by a machine. The most intriguing and sometimes surprising beautiful art is made by non-sentient robots, based on data and interpretations of that data. So what are artists afraid of?",Ethics and Environment,Technology,Jamill Waters & Jess Abbott,Room 103
3,07/27/2020 02:00 PM,07/27/2020 03:30 PM,Augmented Real(ity) Estate,"Imagine if your company is moving you to a state too far away to spend time looking for a new place to live. Wouldn't it be nice to be able to meet an agent, walk through a house, open doors, go up steps, and check out the neighborhood from the comfort of your couch? Check out the latest innovations in augmented reality in the real estate market, and discuss its economic benefits.",Virtual and Augmented Reality,Technology,Grant Jacobson,Room 104
4,07/27/2020 02:00 PM,07/27/2020 03:00 PM,Hands-On Hacks,Join your fellow makers in learning their favorite hacks in popular maker categories.,Education and Training,Education,David Powlowski,Grand View Hall


In [8]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path=file_name)
docs = loader.load()

In [9]:
len(docs)

30

In [10]:
print(docs[0].page_content[:500])

Start Date: 07/27/2020 02:00 PM
End Date: 07/27/2020 03:30 PM
Session Name: 3D Printing for the Non-Tech Minded
Session Description: This is 3D Printing 101 for those in the makerspace that don’t consider themselves technically astute.
Session Track: 3D Printing and Design
Industry: Technology
Speaker Name: Jeffery Lowe & Marysa Balma
Room Name: Room 101


In [11]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

chunk_size = 128
chunk_overlap = 32

In [12]:
c_text_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    separator = ' '
)

r_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
    add_start_index = True,
)

pages = c_text_splitter.split_documents(docs)

In [13]:
print(docs[0])
print(pages[0])
print(pages[1])

page_content='Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:30 PM\nSession Name: 3D Printing for the Non-Tech Minded\nSession Description: This is 3D Printing 101 for those in the makerspace that don’t consider themselves technically astute.\nSession Track: 3D Printing and Design\nIndustry: Technology\nSpeaker Name: Jeffery Lowe & Marysa Balma\nRoom Name: Room 101' metadata={'source': 'conference_session_info.csv', 'row': 0}
page_content='Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:30 PM\nSession Name: 3D Printing for the Non-Tech Minded\nSession' metadata={'source': 'conference_session_info.csv', 'row': 0}
page_content='for the Non-Tech Minded\nSession Description: This is 3D Printing 101 for those in the makerspace that don’t consider themselves' metadata={'source': 'conference_session_info.csv', 'row': 0}


In [14]:
pages = r_text_splitter.split_documents(docs)

print(docs[0])
print(pages[0])
print(pages[1])

page_content='Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:30 PM\nSession Name: 3D Printing for the Non-Tech Minded\nSession Description: This is 3D Printing 101 for those in the makerspace that don’t consider themselves technically astute.\nSession Track: 3D Printing and Design\nIndustry: Technology\nSpeaker Name: Jeffery Lowe & Marysa Balma\nRoom Name: Room 101' metadata={'source': 'conference_session_info.csv', 'row': 0}
page_content='Start Date: 07/27/2020 02:00 PM\nEnd Date: 07/27/2020 03:30 PM\nSession Name: 3D Printing for the Non-Tech Minded' metadata={'source': 'conference_session_info.csv', 'row': 0, 'start_index': 0}
page_content='Session Description: This is 3D Printing 101 for those in the makerspace that don’t consider themselves technically astute.' metadata={'source': 'conference_session_info.csv', 'row': 0, 'start_index': 112}


In [15]:
print(len(docs))
print(len(pages))

30
155


In [16]:
!pip install chromadb

In [17]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [18]:
persist_directory = 'persist_chroma'

In [31]:
newpages = pages[:1]
print(len(newpages))

1


In [ ]:
vectordb = Chroma.from_documents(
    documents=newpages,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

In [ ]:
vectordb.persist()

In [ ]:
question = "which sessions are about augmented reality?"
docs = vectordb.similarity_search(question, k=3)

In [ ]:
docs